# GoogLeNet
(without LRN)

In [ ]:
#full architecture

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/GoogleNet.PNG?versionId=CAEQFRiBgIDNw62XyhciIDA0ZDViOWM5YjYyNDRlZmJiMmMzNWVjOTZlNjk5YmMw)

In [ ]:
#conv + BN + ReLU -- basicconv
#Inception
#AUXclf

In [1]:
import torch
from torch import nn
from torchinfo import summary

In [10]:
class BasicConv2d(nn.Module):
    def __init__(self,in_channels, out_channels,**kwargs
                ):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
                                 ,nn.BatchNorm2d(out_channels)
                                 ,nn.ReLU(inplace=True))
    def forward(self,x):
        x = self.conv(x)
        return x

In [11]:
BasicConv2d(2,10,kernel_size=3)

BasicConv2d(
  (conv): Sequential(
    (0): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
)

In [56]:
class Inception(nn.Module):
    def __init__(self
                 ,in_channels : int
                 ,ch1x1 : int
                 ,ch3x3red : int
                 ,ch3x3 : int
                 ,ch5x5red : int
                 ,ch5x5 : int
                 ,pool_proj : int
                ):
        super().__init__()
        #1x1
        self.branch1 = BasicConv2d(in_channels,ch1x1,kernel_size=1)
        #1x1 + 3x3
        self.branch2 = nn.Sequential(BasicConv2d(in_channels, ch3x3red, kernel_size=1)
                                     ,BasicConv2d(ch3x3red, ch3x3, kernel_size=3,padding=1))
        #1x1 + 5x5
        self.branch3 = nn.Sequential(BasicConv2d(in_channels, ch5x5red, kernel_size=1)
                                     ,BasicConv2d(ch5x5red, ch5x5, kernel_size=5, padding=2))
        #pool + 1x1
        self.branch4 = nn.Sequential(nn.MaxPool2d(kernel_size=3,stride=1, padding=1,ceil_mode=True)
                                    ,BasicConv2d(in_channels,pool_proj,kernel_size=1))
    def forward(self,x):
        branch1 = self.branch1(x) #28x28,ch1x1
        branch2 = self.branch2(x) #28x28,ch3x3
        branch3 = self.branch3(x) #28x28,ch5x5
        branch4 = self.branch4(x) #28x28,pool_proj
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1) #合并

In [ ]:
#(28x28, ch1x1+ ch3x3 + ch5x5 + pool_proj)

In [19]:
a = torch.ones(2,5)

In [20]:
a

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [22]:
b = torch.ones(3,5)

In [23]:
b

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [24]:
(2,9) #不能合并 (4,4&5)

((2, 9),)

In [ ]:
#横向不能合并(2 & 3)
(5,5)

In [26]:
torch.cat([a,b],dim=0)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [28]:
#测试
'''
in_channels : int
,ch1x1 : int
,ch3x3red : int
,ch3x3 : int
,ch5x5red : int
,ch5x5 : int
,pool_proj : int
'''

In [30]:
in3a = Inception(192,64,96,128,16,32,32)

In [42]:
data = torch.ones(10,192,28,28)

In [44]:
in3a(data).shape

torch.Size([10, 256, 28, 28])

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/GoogleNet.PNG?versionId=CAEQFRiBgIDNw62XyhciIDA0ZDViOWM5YjYyNDRlZmJiMmMzNWVjOTZlNjk5YmMw)

In [ ]:
#auxiliary classifier

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/辅助分类器.PNG?versionId=CAEQFRiBgIC7w62XyhciIGNkN2E5ZGRiMjlmZTQxYzY5YmY1ODA3YWI0YjYyNDYx)

In [45]:
class AuxClf(nn.Module):
    def __init__(self,in_channels : int, num_classes : int, **kwargs):
        super().__init__()
        self.feature_ = nn.Sequential(nn.AvgPool2d(kernel_size=5,stride=3)
                                     ,BasicConv2d(in_channels,128, kernel_size=1))
        self.clf_ = nn.Sequential(nn.Linear(4*4*128, 1024)
                                 ,nn.ReLU(inplace=True)
                                 ,nn.Dropout(0.7)
                                 ,nn.Linear(1024,num_classes))
    def forward(self,x):
        x = self.feature_(x)
        x = x.view(-1,4*4*128)
        x = self.clf_(x)
        return x

In [46]:
#4a后的辅助分类器
AuxClf(512,1000)

AuxClf(
  (feature_): Sequential(
    (0): AvgPool2d(kernel_size=5, stride=3, padding=0)
    (1): BasicConv2d(
      (conv): Sequential(
        (0): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (clf_): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.7, inplace=False)
    (3): Linear(in_features=1024, out_features=1000, bias=True)
  )
)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/GoogleNet.PNG?versionId=CAEQFRiBgIDNw62XyhciIDA0ZDViOWM5YjYyNDRlZmJiMmMzNWVjOTZlNjk5YmMw)

In [54]:
#(224 + 6 - 7)/2 + 1 = 112.5
#pool (112 - 3)/2 + 1 = 55.5 向上取整之后得到56

In [60]:
class GoogLeNet(nn.Module):
    def __init__(self,num_classes: int = 1000, blocks = None):
        super().__init__()
        
        if blocks is None:
            blocks = [BasicConv2d, Inception, AuxClf]
        conv_block = blocks[0]
        inception_block = blocks[1]
        aux_clf_block = blocks[2]
        
        #block1
        self.conv1 = conv_block(3,64,kernel_size=7,stride=2,padding = 3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3,stride=2,ceil_mode = True)
        
        #block2
        self.conv2 = conv_block(64,64,kernel_size=1)
        self.conv3 = conv_block(64,192,kernel_size=3, padding = 1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3,stride=2,ceil_mode = True)
        
        #block3
        self.inception3a = inception_block(192,64,96,128,16,32,32)
        self.inception3b = inception_block(256,128,128,192,32,96,64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3,stride=2,ceil_mode = True)

        
        #block4 
        self.inception4a = inception_block(480,192,96,208,16,48,64)
        self.inception4b = inception_block(512,160,112,224,24,64,64)
        self.inception4c = inception_block(512,128,128,256,24,64,64)
        self.inception4d = inception_block(512,112,144,288,32,64,64)
        self.inception4e = inception_block(528,256,150,320,32,128,128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3,stride=2,ceil_mode = True)
        
        #block5
        self.inception5a = inception_block(832,256,160,320,32,128,128)
        self.inception5b = inception_block(832,384,192,384,48,128,128)
        
        #clf
        self.avgpool = nn.AdaptiveAvgPool2d((1,1)) #我需要的输出的特征图尺寸是多少
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024,num_classes)
        
        #auxclf
        self.aux1 = aux_clf_block(512, num_classes) #4a
        self.aux2 = aux_clf_block(528, num_classes) #4d
    
    def forward(self,x):
        #block1
        x = self.maxpool1(self.conv1(x))
        
        #block2
        x = self.maxpool2(self.conv3(self.conv2(x)))
        
        #block3
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        #block4
        x = self.inception4a(x)
        aux1 = self.aux1(x)
        
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        aux2 = self.aux2(x)
        
        x = self.inception4e(x)
        x = self.maxpool4(x)
        
        #block5
        x = self.inception5a(x)
        x = self.inception5b(x)
        
        #clf
        x = self.avgpool(x) #在这个全局平均池化之后，特征图尺寸就变成了1x1
        x = torch.flatten(x,1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x, aux2, aux1

In [ ]:
#测试

In [58]:
data = torch.ones(10,3,224,224)

In [59]:
net = GoogLeNet(num_classes=1000)

In [61]:
fc2, fc1, fc0 = net(data)

In [62]:
for i in [fc2, fc1, fc0]:
    print(i.shape)

torch.Size([10, 1000])
torch.Size([10, 1000])
torch.Size([10, 1000])


In [64]:
summary(net,(10,3,224,224),device="cpu",depth=1)

Layer (type:depth-idx)                   Output Shape              Param #
├─BasicConv2d: 1-1                       [10, 64, 112, 112]        9,536
├─MaxPool2d: 1-2                         [10, 64, 56, 56]          --
├─BasicConv2d: 1-3                       [10, 64, 56, 56]          4,224
├─BasicConv2d: 1-4                       [10, 192, 56, 56]         110,976
├─MaxPool2d: 1-5                         [10, 192, 28, 28]         --
├─Inception: 1-6                         [10, 256, 28, 28]         164,064
├─Inception: 1-7                         [10, 480, 28, 28]         389,376
├─MaxPool2d: 1-8                         [10, 480, 14, 14]         --
├─Inception: 1-9                         [10, 512, 14, 14]         376,800
├─AuxClf: 1-10                           [10, 1000]                3,188,968
├─Inception: 1-11                        [10, 512, 14, 14]         449,808
├─Inception: 1-12                        [10, 512, 14, 14]         510,768
├─Inception: 1-13                        [

# ResNet

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/ResNet架构.PNG?versionId=CAEQFRiBgIDQw62XyhciIDUzZmRhZGJhNThjNTQ3NzhhZjAxOGFjYWM1MjYxZmMw)

In [ ]:
#basicconv - conv2d + BN + ReLU (→ conv3x3, conv1x1)
#Residual Unit, Bottleneck

In [1]:
#导入需要的库
import torch
import torch.nn as nn
from typing import Type, Union, List, Optional
from torchinfo import summary

In [12]:
def conv3x3(in_, out_, stride=1, initialzero = False):
    bn = nn.BatchNorm2d(out_)
    #需要进行判断：要对BN进行0初始化吗？
    #最后一层就初始化,不是最后一层就不改变gamma和beta
    if initialzero == True:
        nn.init.constant_(bn.weight, 0)
    return nn.Sequential(nn.Conv2d(in_, out_
                            , kernel_size=3,padding=1, stride = stride
                            , bias = False)
                         ,bn)

In [13]:
conv3x3(2,10)

Sequential(
  (0): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [14]:
def conv1x1(in_, out_, stride=1, initialzero = False):
    bn = nn.BatchNorm2d(out_)
    #需要进行判断：要对BN进行0初始化吗？
    #最后一层就初始化,不是最后一层就不改变gamma和beta
    if initialzero == True:
        nn.init.constant_(bn.weight, 0)
    return nn.Sequential(nn.Conv2d(in_, out_
                            , kernel_size=1,padding=0, stride = stride
                            , bias = False)
                         ,bn)

In [17]:
conv1x1(2,10,1,True)[1].weight #请帮我执行0初始化

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [18]:
conv1x1(2,10,1)[1].weight #没有执行0初始化

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], requires_grad=True)

In [78]:
class ResidualUnit(nn.Module):
    #这是残差单元类
    #stride1是否等于2呢？如果等于2 - 特征图尺寸会发生变化
    #需要在跳跃链接上增加1x1卷积层来调整特征图尺寸
    #如果stride1等于1，则什么也不需要做
    def __init__(self,out_: int
                 ,stride1: int = 1 #定义该参数的类型，并且定义默认值
                 ,in_ : Optional[int] = None
                ):
        super().__init__()
        
        self.stride1 = stride1
        
        #当特征图尺寸需要缩小时，卷积层的输出特征图数量out_等于输入特征图数量in_的2被
        #当特征图尺寸不需要缩小时，out_ == in_
        if stride1 !=1:
            in_ = int(out_/2)
        else:
            in_ = out_
        
        #拟合部分，输出F(x)
        self.fit_ = nn.Sequential(conv3x3(in_,out_,stride=stride1)
                                 ,nn.ReLU(inplace=True)
                                 ,conv3x3(out_,out_,initialzero=True)
                                 )
        
        #跳跃链接，输出x(1x1卷积核之后的x)
        self.skipconv = conv1x1(in_,out_,stride = stride1)
        
        #单独定义放在H(x)之后来使用的激活函数ReLU
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self,x):
        fx = self.fit_(x) #拟合结果
        if self.stride1 != 1:
            x = self.skipconv(x) #跳跃链接
        hx = self.relu(fx + x)
        return hx

In [ ]:
#ResidualUnit(out_,stride1)

In [22]:
data = torch.ones(10,64,56,56)

In [23]:
conv3_x_18_0 = ResidualUnit(out_=128,stride1 = 2)
#0号残差单元 - 需要特征图这班，特征图数量加倍

In [25]:
conv3_x_18_0(data).shape

torch.Size([10, 128, 28, 28])

In [26]:
conv2_x_18_0 = ResidualUnit(out_ = 64)

In [28]:
conv2_x_18_0(data).shape

torch.Size([10, 64, 56, 56])

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/35.PNG?versionId=CAEQFRiBgID07vWryhciIDRhYTkwODA1MDU0YjRmNWQ5ZTk2NzAxMjdkYzE2MGZm)

In [29]:
class Bottleneck(nn.Module):
    #是需要将特征图尺寸缩小的场合吗？
    #conv2_x - conv3_x - conv4_x - conv5_x 相互链接的时候
    #每次都需要将特征图尺寸折半，同时卷积层上的middle_out = 1/2in_
    def __init__(self, middle_out
                 , stride1: int = 1
                 , in_: Optional[int] = None):
        super().__init__()
        
        out_ = 4 * middle_out
        
        #我希望使用选填参数in_来帮助我们区别，这个架构是不是在conv1的后面
        #如果这个架构不是紧跟在conv1后，就不填写in_
        #如果是跟在conv1后，就填写in_ = 64
        if in_ == None:
            if stride1 !=1: #缩小特征图的场合，即这个瓶颈结构是每个layers的第一个瓶颈结构
                in_ = middle_out * 2
                #不缩小特征图的场合，即这个瓶颈结构不是这个layers的第一个瓶颈结构
                #而是跟在第一个瓶颈结构后的重复的结构
            else:
                in_ = middle_out * 4
        
        self.fit_ = nn.Sequential(conv1x1(in_,middle_out,stride=stride1)
                                 ,nn.ReLU(inplace=True)
                                 ,conv3x3(middle_out,middle_out)
                                 ,nn.ReLU(inplace=True)
                                 ,conv1x1(middle_out,out_,initialzero=True))
        
        self.skipconv = conv1x1(in_, out_, stride=stride1)
        
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self,x):
        fx = self.fit_(x)
        #跳跃链接
        x = self.skipconv(x)
        hx = self.relu(fx + x)
        return hx

In [ ]:
#测试

In [32]:
data1 = torch.ones(10,64,56,56) #conv2x的输入
#假设，我是conv1后紧跟的第一个瓶颈结构

In [30]:
conv2_x_101_0 = Bottleneck(in_ = 64, middle_out=64)

In [33]:
conv2_x_101_0(data1).shape #特征图尺寸不变，输出翻四倍

torch.Size([10, 256, 56, 56])

In [ ]:
#不是conv1后紧跟的第一个瓶颈结构，但是需要缩小特征图尺寸

In [34]:
data2= torch.ones(10,256,56,56)

In [35]:
conv3_x_101_0 = Bottleneck(middle_out=128,stride1=2)

In [36]:
conv3_x_101_0(data2).shape #输出翻两倍，特征图尺寸缩小一半

torch.Size([10, 512, 28, 28])

In [ ]:
#不是conv1后的第一个瓶颈结构，也不需要缩小特征图

In [38]:
data3 = torch.ones(10,512,28,28)
conv3_x_101_1 = Bottleneck(128)

In [39]:
conv3_x_101_1(data3).shape #输出数量不变，特征图尺寸也不变

torch.Size([10, 512, 28, 28])

class Bottleneck(middle_out, stride1, in_(optional))
class ResidualUnit(out_, stride1)

In [47]:
#在单一的layers里的残差单元/瓶颈结构的数量使用num_blocks来表示

In [48]:
#34
num_blocks_conv3x = 4

In [ ]:
#34layer - conv3_x
ru0 = ResidualUnit(out_ = 128,stride1=2) #第一层有点不同，后面的全靠循环

In [49]:
for i in range(num_blocks_conv3x - 1):
    print(ResidualUnit(out_ = 128))

ResidualUnit(
  (fit_): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (skipconv): Sequential(
    (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu): ReLU(inplace=True)
)
ResidualUnit(
  (fit_): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): Seque

In [53]:
#350
num_blocks_conv4_x = 6

In [54]:
conv4_x_50 = []

In [55]:
#在列表中添加第0个瓶颈架构块
conv4_x_50.append(Bottleneck(middle_out = 256,stride1 = 2))

for i in range(num_blocks_conv4_x - 1):
    conv4_x_50.append(Bottleneck(middle_out = 256))

In [57]:
len(conv4_x_50) #包含了6个块，第一个块是包含步长=2的卷积层，剩下的块是重复结构

6

In [ ]:
#34-conv2
#50-conv2

In [ ]:
#34-conv2
ru0 = ResidualUnit(out_ = 64) #不改变特征图尺寸
ru1 = ResidualUnit(out_ = 64)
ru2 = ResidualUnit(out_ = 64)

In [ ]:
#50-conv2
bt0 = Bottleneck(middle_out=64, in_ = 64) #也不需要改变特征图尺寸，需要填写参数in_，参数in_ = 64
bt1 = Bottleneck(middle_out=64)
bt2 = Bottleneck(middle_out=64)

In [ ]:
# 第一个块
# if 这个块是conv1之后的第一个块：
#   需要一个独特的参数 in_ = 64
# else:
#   第一层就需要独特的参数 stride1 = 2

# 剩下的块就使用for循环来跑

In [76]:
layers = []
afterconv1 = True #你是conv1之后的第一个块，false = 你不是conv1之后的第一个块
num_blocks = 6

In [73]:
if afterconv1 == True:
    layers.append(Bottleneck(middle_out=64, in_ = 64))
else:
    layers.append(Bottleneck(middle_out=128, stride1 = 2))

for i in range(num_blocks-1):
    layers.append(Bottleneck(middle_out=128))

In [75]:
len(layers)

6

In [79]:
if afterconv1 == True:
    layers.append(ResidualUnit(out_=64, in_ = 64))
else:
    layers.append(ResidualUnit(out_=64, stride1 = 2))

for i in range(num_blocks-1):
    layers.append(ResidualUnit(out_=64))

In [81]:
len(layers)

6

In [ ]:
#同时用于残差单元和瓶颈结构
#帮助我们将一个layers内的全部块都打包在一个列表中

In [ ]:
#专门用来生成ResNet的每一个layers的函数

In [ ]:
#bool
#int
#float

In [93]:
def make_layers(block: Type[Union[ResidualUnit, Bottleneck]]
                ,middle_out: int
                ,num_blocks: int
                ,afterconv1: bool = False):
    
    layers = []
    
    if afterconv1 == True:
        layers.append(block(middle_out, in_ = 64))
    else:
        layers.append(block(middle_out, stride1 = 2))
    
    for i in range(num_blocks-1):
        layers.append(block(middle_out))
    
    return nn.Sequential(*layers)

In [87]:
layer_34_conv4_x = make_layers(ResidualUnit,
                              256,
                              6,
                              False)

In [92]:
nn.Sequential(*layer_34_conv4_x) #python常见用法，星号解析列表/储存器

Sequential(
  (0): ResidualUnit(
    (fit_): Sequential(
      (0): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ReLU(inplace=True)
      (2): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (skipconv): Sequential(
      (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): ReLU(inplace=True)
  )
  (1): ResidualUnit(
    (fit_): Sequential(
      (0): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_

#测试 - 需要分别对残差块和瓶颈架构进行测试，并且需要对conv1后的首个架构，以及中间的架构进行测试<br>
#注意检查：输入的数据结构是否正确，网络能否允许正确的数据结构输入，输入后产出的结构是否正确，包括特征图尺寸是否变化、特征图数量是否变化，以及一个layers中所包含的blocks数量是否正确

In [ ]:
#34层网络，conv2_x，紧跟在conv1后的首个架构
#不缩小特征图尺寸，每层的输出都是64，3个块

In [100]:
conv2_x_34 = make_layers(ResidualUnit,
           64,
           3,
           afterconv1 = True)

In [101]:
datashape = (10,64,56,56)

In [102]:
summary(conv2_x_34,datashape,depth=1,device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
├─ResidualUnit: 1-1                      [10, 64, 56, 56]          78,208
├─ResidualUnit: 1-2                      [10, 64, 56, 56]          78,208
├─ResidualUnit: 1-3                      [10, 64, 56, 56]          78,208
Total params: 234,624
Trainable params: 234,624
Non-trainable params: 0
Total mult-adds (M): 291.59
Input size (MB): 8.03
Forward/backward pass size (MB): 0.00
Params size (MB): 0.94
Estimated Total Size (MB): 8.97

In [104]:
datashape = (10,64,56,56)

In [103]:
conv2_x_101 = make_layers(Bottleneck,
           64,
           3,
           afterconv1 = True)

In [107]:
summary(conv2_x_101,datashape,depth=3,device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
├─Bottleneck: 1-1                        [10, 256, 56, 56]         --
|    └─Sequential: 2-1                   [10, 256, 56, 56]         --
|    |    └─Sequential: 3-1              [10, 64, 56, 56]          4,224
|    |    └─ReLU: 3-2                    [10, 64, 56, 56]          --
|    |    └─Sequential: 3-3              [10, 64, 56, 56]          36,992
|    |    └─ReLU: 3-4                    [10, 64, 56, 56]          --
|    |    └─Sequential: 3-5              [10, 256, 56, 56]         16,896
|    └─Sequential: 2-2                   [10, 256, 56, 56]         --
|    |    └─Conv2d: 3-6                  [10, 256, 56, 56]         16,384
|    |    └─BatchNorm2d: 3-7             [10, 256, 56, 56]         512
|    └─ReLU: 2-3                         [10, 256, 56, 56]         --
├─Bottleneck: 1-2                        [10, 256, 56, 56]         --
|    └─Sequential: 2-4                   [10, 256, 56, 56]         --

In [108]:
conv4_x_101 = make_layers(Bottleneck,256,23)

In [109]:
datashape = (10,512,28,28)

In [112]:
summary(conv4_x_101, datashape, depth=1, device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
├─Bottleneck: 1-1                        [10, 1024, 14, 14]        1,512,448
├─Bottleneck: 1-2                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-3                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-4                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-5                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-6                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-7                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-8                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-9                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-10                       [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-11                       [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-12                       [10, 1024, 14, 14]        2,167,808
├

In [ ]:
(224 +2p - 7)/2 = 112 #p = 3

In [ ]:
layers18 = [2,2,2,2] #列举了各层分别有多少个块的列表
layer101 = [3,4,23,3]

In [114]:
class ResNet(nn.Module):
    def __init__(self,block: Type[Union[ResidualUnit, Bottleneck]]
                ,layers: List[int]
                ,num_classes : int):
        super().__init__()
        
        '''
        block：要使用的用来加深深度的基本架构是？可以选择残差单元或瓶颈结构，两种都带有skip connection
        layers：列表，每个层里具体有多少个块呢？可参考网络架构图。例如，34层的残差网络的layers = [3,4,6,3]
        num_classes：真实标签含有多少个类别？
        '''
        
        #layer1:卷积+池化的组合
        self.layer1 = nn.Sequential(nn.Conv2d(3,64
                                              ,kernel_size=7,stride=2
                                              ,padding=3,bias = False)
                                   ,nn.BatchNorm2d(64)
                                   ,nn.ReLU(inplace=True)
                                   ,nn.MaxPool2d(kernel_size=3
                                                 ,stride=2
                                                 ,ceil_mode = True))
        
        #layer2 - layer5:残差块/瓶颈结构
        self.layer2_x = make_layers(block,64,layers[0],afterconv1=True)
        self.layer3_x = make_layers(block,128,layers[1])
        self.layer4_x = make_layers(block,256,layers[2])
        self.layer5_x = make_layers(block,512,layers[3])
        
        #全局平均池化
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        
        #分类
        if block == ResidualUnit:
            self.fc = nn.Linear(512,num_classes)
        else:
            self.fc = nn.Linear(2048,num_classes)
            
    def forward(self,x):
        x = self.layer1(x) #layer1，普通卷积+池化的输出
        x = self.layer5_x(self.layer4_x(self.layer3_x(self.layer2_x(x))))
        x = self.avgpool(x) #特征图尺寸1x1 (n_samples, fc, 1, 1)
        x = torch.flatten(x,1)
        x = self.fc(x)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/ResNet架构.PNG?versionId=CAEQFRiBgIDQw62XyhciIDUzZmRhZGJhNThjNTQ3NzhhZjAxOGFjYWM1MjYxZmMw)

In [ ]:
#ResNet(block,layers,num_classes)

In [ ]:
datashape = (10,3,224,224) #ImageNet数据集的结构

In [116]:
res34 = ResNet(ResidualUnit,[3,4,6,3],num_classes = 1000)

In [117]:
res101 = ResNet(Bottleneck,[3,4,23,3],num_classes = 1000)

In [120]:
summary(res34,datashape,depth=2,device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [10, 64, 56, 56]          --
|    └─Conv2d: 2-1                       [10, 64, 112, 112]        9,408
|    └─BatchNorm2d: 2-2                  [10, 64, 112, 112]        128
|    └─ReLU: 2-3                         [10, 64, 112, 112]        --
|    └─MaxPool2d: 2-4                    [10, 64, 56, 56]          --
├─Sequential: 1-2                        [10, 64, 56, 56]          --
|    └─ResidualUnit: 2-5                 [10, 64, 56, 56]          78,208
|    └─ResidualUnit: 2-6                 [10, 64, 56, 56]          78,208
|    └─ResidualUnit: 2-7                 [10, 64, 56, 56]          78,208
├─Sequential: 1-3                        [10, 128, 28, 28]         --
|    └─ResidualUnit: 2-8                 [10, 128, 28, 28]         230,144
|    └─ResidualUnit: 2-9                 [10, 128, 28, 28]         312,064
|    └─ResidualUnit: 2-10                [10, 128, 28, 28] 

In [121]:
summary(res101,datashape,depth=2,device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [10, 64, 56, 56]          --
|    └─Conv2d: 2-1                       [10, 64, 112, 112]        9,408
|    └─BatchNorm2d: 2-2                  [10, 64, 112, 112]        128
|    └─ReLU: 2-3                         [10, 64, 112, 112]        --
|    └─MaxPool2d: 2-4                    [10, 64, 56, 56]          --
├─Sequential: 1-2                        [10, 256, 56, 56]         --
|    └─Bottleneck: 2-5                   [10, 256, 56, 56]         75,008
|    └─Bottleneck: 2-6                   [10, 256, 56, 56]         136,448
|    └─Bottleneck: 2-7                   [10, 256, 56, 56]         136,448
├─Sequential: 1-3                        [10, 512, 28, 28]         --
|    └─Bottleneck: 2-8                   [10, 512, 28, 28]         379,392
|    └─Bottleneck: 2-9                   [10, 512, 28, 28]         543,232
|    └─Bottleneck: 2-10                  [10, 512, 28, 28